# Modeling human metabolism based on Rhea reactions

Rhea was not designed for modeling purposes, however, since it is the reference reaction vocabulary for SwissProt and UniProt, it is possible to build draft models based on combination of Rhea data processing and SPARQL query to Rhea and UniProt

1. Install pyrheadb package

In [ ]:
#!pip install -e ../

2. Import necessary modules of pyrheadb

In [ ]:
import os
import pandas as pd
from pyrheadb.RheaDB import RheaDB
from pyrheadb.RheaCobraModel import RheaCobraModel
from pyrheadb import RheaSPARQL

3. Request only the reactions of human

In [ ]:
os.makedirs('cache', exist_ok=True)

In [ ]:
file_human_reactions = os.path.join('cache', 'df_human_reactions.tsv')
if not os.path.exists(file_human_reactions):
    df_human_reactions = RheaSPARQL.get_all_human_reactions()
    df_human_reactions.to_csv(file_human_reactions, sep='\t', index=False)
df_human_reactions = pd.read_csv(file_human_reactions, sep='\t')
df_human_reactions

4. Filter RheaDB (rdb) table used in this analysis to keep only the human reactions

In [ ]:
df_human_reactions.drop_duplicates(subset=['MASTER_ID'], inplace=True)
rdb_human = RheaDB()
rdb_human.df_reactions = \
                rdb_human.df_reactions.merge(df_human_reactions['MASTER_ID'], how='inner', on='MASTER_ID')

In [ ]:
rhea_human_model = RheaCobraModel('rhea_human_model', rdb_human)

In [ ]:
rhea_human_model.inspect_model()

In [ ]:
# # Write down model reactions if you want to inspect it in more detail
# with open(os.path.join('cache','model_reactions.txt'), 'w') as w:
#     for r in rhea_model.model.reactions:
#         w.write(f"{r}\n")

# Use cobra to analyse consistency of fluxes
Text extract added from cobra doc for clarity (https://cobrapy.readthedocs.io/en/latest/consistency.html)

In [ ]:
from cobra import flux_analysis

## Using FVA

The first approach we can follow is to use FVA (Flux Variability Analysis) which among many other applications, is used to detect blocked reactions. The cobra.flux_analysis.find_blocked_reactions() function will return a list of all the blocked reactions obtained using FVA.

In [ ]:
%time
blocked_reactions=flux_analysis.find_blocked_reactions(rhea_human_model.model)

In [ ]:
print(len(blocked_reactions), 'blocked reactions')

## Using FASTCC
The second approach to obtaining consistent network in cobrapy is to use FASTCC. Using this method,
you can expect to efficiently obtain an accurate consistent network. For more details regarding the algorithm,
please see Vlassis N, Pacheco MP, Sauter T (2014).

In [ ]:
rhea_human_m = rhea_human_model.model
rhea_human_m.solver = "glpk"

# Lower the model's tolerance so we can set a smaller zero_cutoff
rhea_human_m.tolerance = 1e-9

def shallow_copy(self):
    return self  # WARNING: modifies original model in place

rhea_human_m.copy = shallow_copy.__get__(rhea_human_m)
consistent_human_model = flux_analysis.fastcc(rhea_human_m, flux_threshold=1.0, zero_cutoff=1e-9)

print("Number of consistent reactions according to FASTCC:",
      len(consistent_human_model.reactions))

## Use cobra to find the value of the flux for the objective reaction

In [ ]:
rhea_human_model.model.objective = 'RHEA_13065' # ATP maintenance

In [ ]:
rhea_human_model.model.optimize().objective_value

# Repeat analysis for the whole RheaDB

In [ ]:
rdb_all = RheaDB()
rhea_all_model = RheaCobraModel('rhea_all_model', rdb_all)

In [ ]:
rhea_all_model.inspect_model()

In [ ]:
%%time
blocked_reactions=flux_analysis.find_blocked_reactions(rhea_all_model.model)
print(len(blocked_reactions), 'blocked reactions')

In [ ]:
%%time
rhea_all_m = rhea_all_model.model
rhea_all_m.solver = "glpk"

# Lower the model's tolerance so we can set a smaller zero_cutoff
rhea_all_m.tolerance = 1e-9

def shallow_copy(self):
    return self  # WARNING: modifies original model in place

rhea_all_m.copy = shallow_copy.__get__(rhea_all_m)
consistent_all_model = flux_analysis.fastcc(rhea_all_m, flux_threshold=1.0, zero_cutoff=1e-9)

print("Number of consistent reactions in all rhea according to FASTCC:",
      len(consistent_all_model.reactions))

In [ ]:
import cobra
print(cobra.__version__)